In [1]:
"""
Purpose: This form of skeletonization doesn't
filter away everything but the biggest remaining mesh but instead:
1) Breaks down mesh into disconnected pieces
2) Saves all significant pieces in array
3) For each significant piece:
a. Runs pymeshfix
b. Runs skeletonization
c. Adds skeleton edges to total skeleton

4) At end stitches the skeleton together


"""

"\nPurpose: This form of skeletonization doesn't\nfilter away everything but the biggest remaining mesh but instead:\n1) Breaks down mesh into disconnected pieces\n2) Saves all significant pieces in array\n3) For each significant piece:\na. Runs pymeshfix\nb. Runs skeletonization\nc. Adds skeleton edges to total skeleton\n\n4) At end stitches the skeleton together\n\n\n"

In [2]:
"""
Will generate the skeletons for all of the
Exhitatory Neurons and Orphan Neurons

Process: 
1) Check which table the neuron is in
2) Filter away any error labels 
3) Run pymeshfix on neuron
4) Run skeletonization
5) Write to datajoint as array

"""

'\nWill generate the skeletons for all of the\nExhitatory Neurons and Orphan Neurons\n\nProcess: \n1) Check which table the neuron is in\n2) Filter away any error labels \n3) Run pymeshfix on neuron\n4) Run skeletonization\n5) Write to datajoint as array\n\n'

In [3]:
import numpy as np
import datajoint as dj
import time
import pymeshfix
import os
import datetime
import calcification_Module as cm
from meshparty import trimesh_io

#for supressing the output
import os, contextlib
import pathlib
import subprocess

#for error counting
from collections import Counter

#for reading in the new raw_skeleton files
import csv

from Skeleton_Stitcher import stitch_skeleton_with_degree_check, find_skeleton_distance

In [25]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 100

schema = dj.schema('microns_pinky')
pinky = dj.create_virtual_module('pinky', 'microns_pinky')


In [5]:
#checking Neurite Skeleton
largest_mesh_perc = pinky.NeuriteSkeleton.fetch("largest_mesh_perc")
np.mean(largest_mesh_perc)

0.9952033151736746

In [6]:
pinky.NeuriteSkeleton()

segmentation segmentation id,segment_id segment id unique within each Segmentation,n_edges number of edges stored,edges array storing edges on each row,n_bodies the amount of segments the neurite was originally split into,largest_mesh_perc the percentage of the entire mesh that the largest submesh makes up
3,648518346341351467,500,=BLOB=,4,0.997983
3,648518346341351503,330,=BLOB=,2,0.999375
3,648518346341351508,489,=BLOB=,82,0.965987
3,648518346341351509,372,=BLOB=,5,0.998435
3,648518346341351514,599,=BLOB=,6,0.998915
3,648518346341351518,399,=BLOB=,5,0.999109
3,648518346341351523,331,=BLOB=,1,1.0
3,648518346341351525,480,=BLOB=,9,0.998368
3,648518346341351526,345,=BLOB=,4,0.999239
3,648518346341351531,355,=BLOB=,3,0.999442


In [7]:
#output for the skeleton edges to be stored by datajoint
""" OLD WAY THAT DATAJOINT WAS GETTING MAD AT 
def read_skeleton(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            bones.append(np.array(line.split()[1:], float).reshape(-1, 3))
    return np.array(bones)
"""

""" NEW FLAT LIST WAY"""
#practice reading in dummy skeleton file
def read_skeleton_flat(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            for r in (np.array(line.split()[1:], float).reshape(-1, 3)):
                bones.append(r)
            bones.append([np.nan,np.nan,np.nan])
    return np.array(bones).astype(float)


""" New read function: for adjusted 2 vert skeleton output"""
def read_raw_skeleton(file_path):
    edges = list()
    with open(file_path) as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        for i,row in enumerate(reader):
            v1 = (float(row[1]),float(row[2]),float(row[3]))
            v2 = (float(row[4]),float(row[5]),float(row[6]))
            edges.append((v1,v2))
    return np.array(edges).astype(float)

def read_skeleton_revised(file_path):
    with open(file_path) as f:
        bones = np.array([])
        for line in f.readlines():
            #print(line)
            line = (np.array(line.split()[1:], float).reshape(-1, 3))
            #print(line[:-1])
            #print(line[1:])

            #print(bones.size)
            if bones.size <= 0:
                bones = np.stack((line[:-1],line[1:]),axis=1)
            else:
                bones = np.vstack((bones,(np.stack((line[:-1],line[1:]),axis=1))))
            #print(bones)


    return np.array(bones).astype(float)



In [8]:
#make sure there is a temp file in the directory, if not then make one
#if temp folder doesn't exist then create it
if (os.path.isdir(os.getcwd() + "/pymesh_NEURITES")) == False:
    os.mkdir("pymesh_NEURITES")

In [9]:
#create the output file
##write the OFF file for the neuron
import pathlib
def write_Whole_Neuron_Off_file(neuron_ID,
                                vertices=[], 
                                triangles=[],
                                folder="pymesh_NEURITES"):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    
    #get the current file location
    file_loc = pathlib.Path.cwd() / folder
    filename = "neuron_" + str(neuron_ID)
    path_and_filename = file_loc / filename
    
    #print(file_loc)
    #print(path_and_filename)
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename),str(filename),str(file_loc)

In [10]:
def meshlab_fix_manifold(key,folder="pymesh_NEURITES"):
    
    file_loc = pathlib.Path.cwd() / folder
    filename = "neuron_" + str(key["segment_id"])
    path_and_filename = str(file_loc / filename)
    
    
    input_mesh = path_and_filename + ".off"
    output_mesh = path_and_filename+"_mls.off"
    
    
    meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
    
    print("starting meshlabserver fixing non-manifolds")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      output_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(key["segment_id"]) + 
                         ' did not fix the manifold edges')
    
    return output_mesh

def meshlab_fix_manifold_path(path_and_filename,segment_id=-1):
    #fix the path if it comes with the extension
    if path_and_filename[-4:] == ".off":
        path_and_filename = path_and_filename[-4:]
    
    input_mesh = path_and_filename + ".off"
    output_mesh = path_and_filename+"_mls.off"
    
    #print("input_mesh = " + str(input_mesh))
    #print("output_mesh = " + str(output_mesh))
    
    meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
    
    print("starting meshlabserver fixing non-manifolds")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      output_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(segment_id) + 
                         ' did not fix the manifold edges')
    
    return output_mesh

In [11]:
def run_meshlab_script(mlx_script,input_mesh_file,output_mesh_file):
    script_command = (" -i " + str(input_mesh_file) + " -o " + 
                                    str(output_mesh_file) + " -s " + str(mlx_script))
    #return script_command
    subprocess_result = subprocess.run('xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + 
                   script_command,shell=True)
    
    return subprocess_result

In [12]:
pinky.Mesh() & pinky.Neurite() & pinky.CurrentSegmentation

segmentation segmentation id,segment_id segment id unique within each Segmentation,n_vertices number of vertices in this mesh,n_triangles number of triangles in this mesh,"vertices x,y,z coordinates of vertices",triangles triangles (triplets of vertices)
3,648518346341351467,28754,57350,=BLOB=,=BLOB=
3,648518346341351503,11192,22282,=BLOB=,=BLOB=
3,648518346341351508,58449,116104,=BLOB=,=BLOB=
3,648518346341351509,24919,49578,=BLOB=,=BLOB=
3,648518346341351514,37788,75374,=BLOB=,=BLOB=
3,648518346341351518,20204,40278,=BLOB=,=BLOB=
3,648518346341351523,15522,30788,=BLOB=,=BLOB=
3,648518346341351525,45961,91644,=BLOB=,=BLOB=
3,648518346341351526,17090,34052,=BLOB=,=BLOB=
3,648518346341351531,21520,42882,=BLOB=,=BLOB=


# Debugging the skeleton stitching function

In [22]:

key = dict(segmentation=3,segment_id=648518346341395072)

split_significance_threshold = 100

global_time = time.time()
#get the mesh with the error segments filtered away
start_time = time.time()
print(str(key['segment_id']) +  ":")
my_dict = (pinky.Mesh & pinky.Neurite.proj() & pinky.CurrentSegmentation
                   & key).fetch1()
print(f"Step 1: Retrieving Mesh and removing error segments: {time.time() - start_time}")
new_key = dict(segmentation=key["segmentation"],
               segment_id=key["segment_id"])


# Don't need these attributes      
#vertices=key["vertices"],
#                       triangles=new_key["triangles"],n_vertices=key["n_vertices"],
#                       n_triangles=key["n_triangles"])


start_time = time.time()
#pass the vertices and faces to pymeshfix to become watertight

mesh = trimesh_io.Mesh(vertices=my_dict["vertices"], faces=my_dict["triangles"])

""" OLDER WAY OF JUST GETTING THE LARGEST MESH PIECE
count, labels = trimesh_io.trimesh.graph.csgraph.connected_components(
                                                    mesh.edges_sparse,
                                                    directed=False,
                                                    return_labels=True)


label_counter = Counter(labels)


new_key["n_bodies"] = count
values = np.array(labels)


list_counter = Counter(labels)
max_counter = max(list_counter.values())

max_label = -1
for label_key,label_number in list_counter.items():
    if label_number==max_counter:
        max_label = label_key
print("max label = " + str(max_label))

searchval = max_label

ii = np.where(values == searchval)[0]
new_key["largest_mesh_perc"] = len(ii)/len(labels)

print("n_bodies = " + str(new_key["n_bodies"]))
print("largest mesh perc = " + str(new_key["largest_mesh_perc"]))
"""



total_splits = mesh.split(only_watertight=False)
print(f"There were {len(total_splits)} after split and significance threshold")
mesh_pieces = [k for k in total_splits if len(k.faces) > split_significance_threshold]
print(f"There were {len(mesh_pieces)} after split and significance threshold")
for g,mh in enumerate(mesh_pieces):
    print(f"Mesh piece {g} with number of faces {len(mh.faces)}")
    
print(f"Step 2a: Getting the number of splits: {time.time() - start_time}")

#get the largest mesh piece
largest_mesh_index = -1
largest_mesh_size = 0

for t,msh in enumerate(mesh_pieces):
    if len(msh.faces) > largest_mesh_size:
        largest_mesh_index = t
        largest_mesh_size = len(msh.faces) 

#largest mesh piece
largest_mesh_perc = largest_mesh_size/len(mesh.faces)
new_key["largest_mesh_perc"] = largest_mesh_perc
print("largest mesh perc = " + str(largest_mesh_perc))

largest_mesh_skeleton_distance = -1

paths_used = []
total_edges = np.array([])
for h,m in enumerate(mesh_pieces): 
    
    print(f"Working on split {h} with face total = {len(m.faces)}")


    start_time = time.time()
    #pass the vertices and faces to pymeshfix to become watertight
    meshfix = pymeshfix.MeshFix(m.vertices,m.faces)
    meshfix.repair(verbose=False,joincomp=True,remove_smallest_components=False)
    print(f"Step 2b: Pymesh shrinkwrapping: {time.time() - start_time}")

    #print("Step 2: Writing Off File")
    start_time = time.time()
    #write the new mesh to off file
    path_and_filename,filename,file_loc = write_Whole_Neuron_Off_file(str(new_key["segment_id"]) + "_piece_" + str(h),meshfix.v,meshfix.f)
    print(f"Step 3: Writing shrinkwrap off file: {time.time() - start_time}")
    #add the path to be deleted later
    paths_used.append(path_and_filename)


    #Run the meshlabserver scripts
    start_time = time.time()
    output_mesh = meshlab_fix_manifold_path(path_and_filename,key["segment_id"])
    print(f"Step 4: Meshlab fixing non-manifolds: {time.time() - start_time}")

    print(output_mesh[:-4])

    #send to be skeletonized
    start_time = time.time()
    return_value = cm.calcification(output_mesh[:-4])
    if return_value > 0:
        raise Exception('skeletonization for neuron ' + str(new_key["segment_id"]) + 
                        ' did not finish... exited with error code: ' + str(return_value))
    #print(f"Step 5: Generating Skeleton: {time.time() - start_time}")



    #read in the skeleton files into an array
    bone_array = read_skeleton_revised(output_mesh[:-4]+"_skeleton.cgal")

    #print(bone_array)
    if len(bone_array) <= 0:
        raise Exception('No skeleton generated for ' + str(new_key["segment_id"]))
    print(f"Step 5: Generating and reading Skeleton: {time.time() - start_time}")

    #get the largest mesh skeleton distance
    if h == largest_mesh_index:
        largest_mesh_skeleton_distance = find_skeleton_distance(bone_array)

    #add the skeleton edges to the total edges
    if not total_edges.any():
        total_edges = bone_array
    else:
        total_edges = np.vstack([total_edges,bone_array])

    

total_edges_stitched = stitch_skeleton_with_degree_check(total_edges)

#get the total skeleton distance for the stitched skeleton
total_skeleton_distance = find_skeleton_distance(total_edges_stitched)

largest_mesh_distance_perc = largest_mesh_skeleton_distance/total_skeleton_distance


start_time = time.time()
new_key["n_edges"] = len(total_edges_stitched)
new_key["edges"] = bone_array
new_key["n_bodies"] = len(total_splits)
new_key["n_bodies_stitched"] = len(mesh_pieces)
new_key["largest_mesh_perc"] = largest_mesh_perc
new_key["largest_mesh_distance_perc"] = largest_mesh_distance_perc

#self.insert1(new_key,skip_duplicates=True)
print(f"Step 6: Inserting dictionary: {time.time() - start_time}")
#raise Exception("done with one neuron")
for path_and_filename in paths_used:
    os.system("rm "+str(path_and_filename)+"*")

print(f"Total time: {time.time() - global_time}")
print("\n\n")

        


648518346341395072:
Step 1: Retrieving Mesh and removing error segments: 0.030570507049560547
There were 6 after split and significance threshold
There were 2 after split and significance threshold
Mesh piece 0 with number of faces 23116
Mesh piece 1 with number of faces 122
Step 2a: Getting the number of splits: 0.0626058578491211
largest mesh perc = 0.9884546309757974
Working on split 0 with face total = 23116
Step 2b: Pymesh shrinkwrapping: 0.49095821380615234
Done writing OFF file
Step 3: Writing shrinkwrap off file: 0.04801154136657715
starting meshlabserver fixing non-manifolds
Step 4: Meshlab fixing non-manifolds: 0.641871452331543
/notebooks/Pass_2_Skeleton_End_Stitiching/pymesh_NEURITES/neuron_648518346341395072_piece_0_mls
Step 5: Generating and reading Skeleton: 0.35820627212524414
Working on split 1 with face total = 122
Step 2b: Pymesh shrinkwrapping: 0.0012402534484863281
Done writing OFF file
Step 3: Writing shrinkwrap off file: 0.001153707504272461
starting meshlabserve

In [27]:
(mesh_pieces[0] + mesh_pieces[1]).show()

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [14]:
#pinky.NeuriteSkeletonStitched.drop()

In [15]:
@schema
class NeuriteSkeletonStitch(dj.Computed):
    definition="""
    -> pinky.Mesh
    ---
    n_edges   :int unsigned #number of edges stored
    edges     :longblob #array storing edges on each row
    n_bodies    :tinyint unsigned #the amount of segments the neurite was originally split into
    n_bodies_stitched  :tinyint unsigned #the amount of segments whose skeletons were stitched back together (aka above the significance threshold)
    largest_mesh_perc : float #number of faces of largest submesh / number of faces of entire skeletal mesh
    largest_mesh_distance_perc : float #skeleton length of largest submesh / skeleton length of entire stitched mesh
    """
    
    
    #key_source = pinky.Mesh() & pinky.Neurite() & pinky.CurrentSegmentation
    key_source = pinky.Mesh() & pinky.Neurite() & pinky.CurrentSegmentation & [dict(segment_id=k) for k in 
                                                                           [648518346341395072, 648518346342916062, 648518346342920252]]
    #how you get the date and time  datetime.datetime.now()
    
    def make(self, key):
        split_significance_threshold = 100

        global_time = time.time()
        #get the mesh with the error segments filtered away
        start_time = time.time()
        print(str(key['segment_id']) +  ":")
        my_dict = (pinky.Mesh & pinky.Neurite.proj() & pinky.CurrentSegmentation
                           & key).fetch1()
        print(f"Step 1: Retrieving Mesh and removing error segments: {time.time() - start_time}")
        new_key = dict(segmentation=key["segmentation"],
                       segment_id=key["segment_id"])


        # Don't need these attributes      
        #vertices=key["vertices"],
        #                       triangles=new_key["triangles"],n_vertices=key["n_vertices"],
        #                       n_triangles=key["n_triangles"])


        start_time = time.time()
        #pass the vertices and faces to pymeshfix to become watertight

        mesh = trimesh_io.Mesh(vertices=my_dict["vertices"], faces=my_dict["triangles"])

        """ OLDER WAY OF JUST GETTING THE LARGEST MESH PIECE
        count, labels = trimesh_io.trimesh.graph.csgraph.connected_components(
                                                            mesh.edges_sparse,
                                                            directed=False,
                                                            return_labels=True)


        label_counter = Counter(labels)


        new_key["n_bodies"] = count
        values = np.array(labels)


        list_counter = Counter(labels)
        max_counter = max(list_counter.values())

        max_label = -1
        for label_key,label_number in list_counter.items():
            if label_number==max_counter:
                max_label = label_key
        print("max label = " + str(max_label))

        searchval = max_label

        ii = np.where(values == searchval)[0]
        new_key["largest_mesh_perc"] = len(ii)/len(labels)

        print("n_bodies = " + str(new_key["n_bodies"]))
        print("largest mesh perc = " + str(new_key["largest_mesh_perc"]))
        """



        total_splits = mesh.split(only_watertight=False)
        print(f"There were {len(total_splits)} after split and significance threshold")
        mesh_pieces = [k for k in total_splits if len(k.faces) > split_significance_threshold]
        print(f"There were {len(mesh_pieces)} after split and significance threshold")
        for g,mh in enumerate(mesh_pieces):
            print(f"Mesh piece {g} with number of faces {len(mh.faces)}")

        print(f"Step 2a: Getting the number of splits: {time.time() - start_time}")

        #get the largest mesh piece
        largest_mesh_index = -1
        largest_mesh_size = 0

        for t,msh in enumerate(mesh_pieces):
            if len(msh.faces) > largest_mesh_size:
                largest_mesh_index = t
                largest_mesh_size = len(msh.faces) 

        #largest mesh piece
        largest_mesh_perc = largest_mesh_size/len(mesh.faces)
        new_key["largest_mesh_perc"] = largest_mesh_perc
        print("largest mesh perc = " + str(largest_mesh_perc))

        largest_mesh_skeleton_distance = -1

        paths_used = []
        total_edges = np.array([])
        for h,m in enumerate(mesh_pieces): 
            print(f"Working on split {h} with face total = {len(m.faces)}")


            start_time = time.time()
            #pass the vertices and faces to pymeshfix to become watertight
            meshfix = pymeshfix.MeshFix(m.vertices,m.faces)
            meshfix.repair(verbose=False,joincomp=True,remove_smallest_components=False)
            print(f"Step 2b: Pymesh shrinkwrapping: {time.time() - start_time}")

            #print("Step 2: Writing Off File")
            start_time = time.time()
            #write the new mesh to off file
            path_and_filename,filename,file_loc = write_Whole_Neuron_Off_file(str(new_key["segment_id"]) + "_piece_" + str(h),meshfix.v,meshfix.f)
            print(f"Step 3: Writing shrinkwrap off file: {time.time() - start_time}")
            #add the path to be deleted later
            paths_used.append(path_and_filename)


            #Run the meshlabserver scripts
            start_time = time.time()
            output_mesh = meshlab_fix_manifold_path(path_and_filename,key["segment_id"])
            print(f"Step 4: Meshlab fixing non-manifolds: {time.time() - start_time}")

            print(output_mesh[:-4])

            #send to be skeletonized
            start_time = time.time()
            return_value = cm.calcification(output_mesh[:-4])
            if return_value > 0:
                raise Exception('skeletonization for neuron ' + str(new_key["segment_id"]) + 
                                ' did not finish... exited with error code: ' + str(return_value))
            #print(f"Step 5: Generating Skeleton: {time.time() - start_time}")



            #read in the skeleton files into an array
            bone_array = read_skeleton_revised(output_mesh[:-4]+"_skeleton.cgal")

            #print(bone_array)
            if len(bone_array) <= 0:
                raise Exception('No skeleton generated for ' + str(new_key["segment_id"]))
            print(f"Step 5: Generating and reading Skeleton: {time.time() - start_time}")

            #get the largest mesh skeleton distance
            if h == largest_mesh_index:
                largest_mesh_skeleton_distance = find_skeleton_distance(bone_array)

            #add the skeleton edges to the total edges
            if not total_edges.any():
                total_edges = bone_array
            else:
                total_edges = np.vstack([total_edges,bone_array])


        total_edges_stitched = stitch_skeleton_with_degree_check(total_edges)

        #get the total skeleton distance for the stitched skeleton
        total_skeleton_distance = find_skeleton_distance(total_edges_stitched)

        largest_mesh_distance_perc = largest_mesh_skeleton_distance/total_skeleton_distance


        start_time = time.time()
        new_key["n_edges"] = len(total_edges_stitched)
        new_key["edges"] = total_edges_stitched
        new_key["n_bodies"] = len(total_splits)
        new_key["n_bodies_stitched"] = len(mesh_pieces)
        new_key["largest_mesh_perc"] = largest_mesh_perc
        new_key["largest_mesh_distance_perc"] = largest_mesh_distance_perc

        self.insert1(new_key,skip_duplicates=True)
        print(f"Step 6: Inserting dictionary: {time.time() - start_time}")
        #raise Exception("done with one neuron")
        for path_and_filename in paths_used:
            os.system("rm "+str(path_and_filename)+"*")

        print(f"Total time: {time.time() - global_time}")
        print("\n\n")
        
        

                         
                                    

In [16]:
#(schema.jobs & "table_name='__neurite_skeleton_stitched'").delete()

In [17]:
start = time.time()
NeuriteSkeletonStitch.populate(reserve_jobs=True)
print(time.time() - start)

648518346341395072:
Step 1: Retrieving Mesh and removing error segments: 0.028834819793701172
There were 6 after split and significance threshold
There were 2 after split and significance threshold
Mesh piece 0 with number of faces 23116
Mesh piece 1 with number of faces 122
Step 2a: Getting the number of splits: 0.13355255126953125
largest mesh perc = 0.9884546309757974
Working on split 0 with face total = 23116
Step 2b: Pymesh shrinkwrapping: 1.4360883235931396
Done writing OFF file
Step 3: Writing shrinkwrap off file: 0.11956167221069336
starting meshlabserver fixing non-manifolds
Step 4: Meshlab fixing non-manifolds: 2.272036075592041
/notebooks/Pass_2_Skeleton_End_Stitiching/pymesh_NEURITES/neuron_648518346341395072_piece_0_mls
Step 5: Generating and reading Skeleton: 0.649385929107666
Working on split 1 with face total = 122
Step 2b: Pymesh shrinkwrapping: 0.002147674560546875
Done writing OFF file
Step 3: Writing shrinkwrap off file: 0.0023474693298339844
starting meshlabserver 

In [18]:
pinky.NeuriteSkeletonStitched()

segmentation segmentation id,segment_id segment id unique within each Segmentation,n_edges number of edges stored,edges array storing edges on each row,n_bodies the amount of segments the neurite was originally split into,n_bodies_stitched the amount of segments whose skeletons were stitched back together (aka above the significance threshold),largest_mesh_perc number of faces of largest submesh / number of faces of entire skeletal mesh,largest_mesh_distance_perc skeleton length of largest submesh / skeleton length of entire stitched mesh
3,648518346341351467,483,=BLOB=,4,1,0.998535,1.0
3,648518346341351503,312,=BLOB=,2,1,0.999551,1.0
3,648518346341351508,557,=BLOB=,82,10,0.968804,0.990936
3,648518346341351509,345,=BLOB=,5,1,0.99879,1.0
3,648518346341351514,564,=BLOB=,6,1,0.999284,1.0
3,648518346341351518,386,=BLOB=,5,1,0.999652,1.0
3,648518346341351523,327,=BLOB=,1,1,1.0,1.0
3,648518346341351525,522,=BLOB=,9,1,0.998909,1.0
3,648518346341351526,331,=BLOB=,4,1,0.999706,1.0
3,648518346341351531,349,=BLOB=,3,1,0.999767,1.0


In [26]:
pinky.NeuriteSkeletonStitched & 'largest_mesh_distance_perc < 0.6'

segmentation segmentation id,segment_id segment id unique within each Segmentation,n_edges number of edges stored,edges array storing edges on each row,n_bodies the amount of segments the neurite was originally split into,n_bodies_stitched the amount of segments whose skeletons were stitched back together (aka above the significance threshold),largest_mesh_perc number of faces of largest submesh / number of faces of entire skeletal mesh,largest_mesh_distance_perc skeleton length of largest submesh / skeleton length of entire stitched mesh
3,648518346341395072,339,=BLOB=,6,2,0.988455,0.572747
3,648518346342916062,914,=BLOB=,14,2,0.525566,0.509383
3,648518346342920252,675,=BLOB=,5,2,0.596253,0.570988
3,648518346349464650,1590,=BLOB=,17,5,0.579346,0.561646
3,648518346349467328,652,=BLOB=,4,2,0.578093,0.567783
3,648518346349468939,1047,=BLOB=,14,3,0.536685,0.557643
3,648518346349471124,1251,=BLOB=,8,2,0.546582,0.542283
3,648518346349472896,860,=BLOB=,54,6,0.567843,0.598852
3,648518346349473586,968,=BLOB=,7,2,0.50054,0.54128
3,648518346349477655,722,=BLOB=,8,2,0.499214,0.50665


# TESTING

In [20]:
# """  Example skeleton that is disconnected """
# example_edges = np.array([
#                           [[0,0,0],[1,0,0]],
#                           [[1,0,0],[1,2,0]],
#                           [[1,2,0],[0,3,0]],
#                           [[0,4,0],[0,4,1]],
#                           [[0,4,1],[0,5,2]],
#                           [[0,5,2],[0,6,-1]],
#                           [[200,200,0.5],[0,6,0.5]],
#                           [[0,6,0.5],[0,15,5]],
#                           [[0,15,5],[100,100,1]]
                          
                          
#                          ])